Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [556]:
from random import choices, randint, choice, random
from functools import reduce
from collections import namedtuple
from dataclasses import dataclass
from copy import copy

from pprint import pprint

import numpy as np

import lab9_lib

In [557]:
POPULATION_SIZE = 100
OFFSPRING_SIZE = 30
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .50

PROBLEM_SIZE = 10
NUM_SETS = 1000

In [558]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)

In [559]:
@dataclass
class Individual:
    fitness: tuple
    genotype: list[bool]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

#MUTATION 3 types
def mutate(ind: Individual) -> Individual:              #STANDARD MUTATION
    offspring = copy(ind)
    pos = randint(0, NUM_SETS-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def n_mutate(ind: Individual, n) -> Individual:         #mutate in n positions (n = random(2,NUM_SETS/10))
    offspring = copy(ind)
    positions = [randint(0, NUM_SETS-1) for _ in range(n)]
    for pos in positions:
        offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def little_reverse(ind: Individual, n) -> Individual:   #reverse a slice of array, n is the lenght of slice(n = random(2,NUM_SETS/10)) 
    offspring = copy(ind)
    pos = randint(0, NUM_SETS-n)
    offspring.genotype[pos: pos+n] = reversed(offspring.genotype[pos: pos+n])
    offspring.fitness = None
    return offspring


#CROSSOVER 3 types
def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:        #STANDARD one cut crossover
    cut_point = randint(0, NUM_SETS-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_SETS
    return offspring

def n_cut_xover(ind1: Individual, ind2: Individual, n: int) -> Individual:  #n cut crossover (n = random(2,NUM_SETS/10))
    cut_points = sorted({0, NUM_SETS-1} | {randint(1, NUM_SETS-2) for _ in range(n)})
    offspring_genotype = []
    current_parent = ind1

    for i, cut_point in enumerate(cut_points):
        end_point = cut_points[i + 1] if i + 1 < len(cut_points) else NUM_SETS
        offspring_genotype.extend(current_parent.genotype[cut_point:end_point])
        current_parent = ind2 if current_parent is ind1 else ind1

    offspring = Individual(fitness=None, genotype=offspring_genotype)
    assert len(offspring.genotype) == NUM_SETS
    return offspring


def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:        #each element is randomly given by the first inidividual or the second
    offspring_genotype = [choice([ind1.genotype[i], ind2.genotype[i]]) for i in range(NUM_SETS)]
    offspring = Individual(fitness=None, genotype=offspring_genotype)
    assert len(offspring.genotype) == NUM_SETS
    return offspring

In [560]:
population = [
    Individual(
        genotype=[choice((0, 1)) for _ in range(NUM_SETS)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)
'''
for ind in population:
    print(f"{''.join(str(g) for g in ind.genotype)}: {ind.fitness:.2%}")
'''

'\nfor ind in population:\n    print(f"{\'\'.join(str(g) for g in ind.genotype)}: {ind.fitness:.2%}")\n'

In [561]:
previous_fitness = 0
same_fintness_counter = 1
for generation in range(1000*PROBLEM_SIZE):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # three types of mutation 
            p = select_parent(population)
            #o = mutate(p)
            o = choice([mutate(p), n_mutate(p, randint(2,int(NUM_SETS/10))), little_reverse(p, randint(2,int(NUM_SETS/10)))])   #randomly selected
        else:
            # three types of xover
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = choice([one_cut_xover(p1, p2), uniform_xover(p1,p2), n_cut_xover(p1,p2,randint(2,int(NUM_SETS/10)))])           #randomly selected
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]           #removes individuals which have the lowest fitness
    print(f"best fitness: {population[0].fitness:.2%} at generation number {generation}, same fitness {same_fintness_counter}")

    if (population[0].fitness==1 or same_fintness_counter > 149*PROBLEM_SIZE):   #stop populating if the fitness is 100% or if there is always the same fitness in 150 population
        break
    if (previous_fitness == population[0].fitness):
        same_fintness_counter = same_fintness_counter + 1
    else :
        previous_fitness = population[0].fitness
        same_fintness_counter = 1
print(f"all fitness call: {fitness.calls}")

best fitness: 16.21% at generation number 0, same fitness 1
best fitness: 16.21% at generation number 1, same fitness 1
best fitness: 17.08% at generation number 2, same fitness 2
best fitness: 17.08% at generation number 3, same fitness 1
best fitness: 17.08% at generation number 4, same fitness 2
best fitness: 17.40% at generation number 5, same fitness 3
best fitness: 17.40% at generation number 6, same fitness 1
best fitness: 17.40% at generation number 7, same fitness 2
best fitness: 17.40% at generation number 8, same fitness 3
best fitness: 17.40% at generation number 9, same fitness 4
best fitness: 17.40% at generation number 10, same fitness 5
best fitness: 17.40% at generation number 11, same fitness 6
best fitness: 21.80% at generation number 12, same fitness 7
best fitness: 21.80% at generation number 13, same fitness 1
best fitness: 21.80% at generation number 14, same fitness 2
best fitness: 21.80% at generation number 15, same fitness 3
best fitness: 21.80% at generation